# Coursework 2: $\ell_1$-Methods for Sparse Recovery

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 2.1 Subgradient Method

### 2.1.0 Differentiable Function

Let $f(\bm{x})$ be a differentiable function. We show that gradient descent direction leads a decrease of the objective function as follows.

The first order Taylor approximation of $f(\bm{x})$ is given by 
$$
    f(\bm{x} + \tau \bm{v}) 
    = f(\bm{x}) 
    + \langle \nabla f(\bm{x}), \tau \bm{v} \rangle 
    + O(\tau^2). 
$$
Take the negative gradient direction, i.e., $\bm{v} = -\nabla f(\bm{x})$. Then
$$
    f(\bm{x} - \tau \nabla f(\bm{x})) 
    = f(\bm{x}) 
    - \tau \| \nabla f(\bm{x}) \|^2_2
    + O(\tau^2). 
$$
It is clearly when $\tau>0$ is sufficiently small, 
$$
    f(\bm{x} -\tau \nabla f(\bm{x})) \le f(\bm{x}),
$$
which proves our claim. 

We study the non-differentiable case below. 

### 2.1.1 Descent Direction? (25%)

Let $\bm{x} \in \mathbb{R}^2$ and 
$$
    f(\bm{x}) = 3 |x_1| +  |x_2|.
$$
Consider the point $\bm{x}^0 = [0,1]^{\mathsf{T}}$. 
1. Show that $\bm{g} = [3,1]^{\mathsf{T}} \in \partial f(\bm{x})$. 
2. Let $\tau \in (0,1)$, find the closed form for 
$$
      f(\bm{x}^0 - \tau \bm{g} ).
$$
3. Comment on whether $-\bm{g}$ is a descent direction or not.

This exercise shows that sub-gradient linear search may not lead to a decrease of the objective function.

### 2.1.2 Wolfe's Example (25%)

Let $\bm{x}\in \mathbb{R}^2$ and 
$$
    f(\bm{x}) = \begin{cases}
        5(9x_1^2 + 16 x_2^2)^{1/2} & \text{if}~ x_1 > |x_2|, \\
        9x_1 + 16|x_2| & \text{if}~ x_1 \le |x_2|.
    \end{cases}
$$
Suppose that $\bm{x}^0 = [16/9,1]^{\mathsf{T}}$. Consider exact line search where 
$$
   \bm{x}^{l+1} = \bm{x}^l - t^{l+1} \nabla f(\bm{x}^l),
$$
where 
$$
   t^{l+1} = \arg~ \min_t~ f(\bm{x}^l - t \nabla f(\bm{x}^l)).
$$
1. Draw the contours of $f(\bm{x})$ in the region $-2 \le x_1 \le 2$ and $-2 \le x_2 \le 2$. (The function `Plots > contour` may be useful.)
2. Is the point $\bm{x} = [0,0]^{\mathsf{T}}$ the global optimal? Why?
3. Find the closed form of $\nabla f(\bm{x})$ in the region where $x_1 > |x_2|$.
4. Find $t^1$ and $\bm{x}^1$ numerically. Note that in the region where $x_1 > |x_2|$
   $$
    \left\langle 
      \nabla f \left(\bm{x}^l+1\right), \nabla f \left(\bm{x}^l\right)
    \right\rangle
    = \left\langle 
      \nabla f \left( \bm{x}^l - t \nabla f \left(\bm{x}^l\right) \right), \nabla f \left(\bm{x}^l\right)
    \right\rangle = 0. 
   $$
5. Similarly, find $t^2$ and $\bm{x}^2$, and $t^3$ and $\bm{x}^3$ numerically. 
6. (Challenging) Use mathematical induction, show that $\bm{x}^l \rightarrow [0,0]^{\mathsf{T}}$.  

Wolfe's example shows that sub-gradient method may not converge to a global optimal point. 

## 2.2 Sparse Recovery: Lasso

### ISTA Implementation and Test (50%)

Implement ISTA algorithm as a function `rec_ista` to solve the Lasso problem
$$
    \frac{1}{2} \| y- Ax \|_2^2 + \lambda \| x \|_1.
$$
where $\lambda > 0$ is the regularization constant. 

Make sure that the regularization constant $\lambda$ and the stepsize $\tau$ are among input parameters.

Suggested parameter for testing: $m=32,~ n=64,~ S=8$. Noise variance can be set small $\sigma^2 = 1e-2 \text{ or } 1e-4$.

1. For a given regularization parameter $\lambda > 0$, try different values of the step size $\tau$, some small compared to $2/\sigma_{\max}(\bm{A}^{\mathsf{T}}\bm{A})$ and some larger than it, for example, $20/\sigma_{\max}(\bm{A}^{\mathsf{T}}\bm{A})$. Discuss the convergence of ISTA when $\tau$ varies.
2. With proper $\tau$ so that ISTA converges, use numerical simulations to discuss how to choose an appropriate $\lambda$ to get reasonable recovery $\hat{x}$.
3. Numerically compare ISTA and SP algorithms for noiseless sparse recovery problem ($\sigma^2 = 0$). Discuss your observations. 

In [1]:
using LinearAlgebra
using SparseArrays
using StatsBase
using JLD2

# Generate Gaussian Matrix
function gen_GaussianMat(m,n)
    #normalize columns
        A = randn(m,n)
        for i = 1:n
            v = A[:,i]
            A[:,i] = v/norm(v,2)
        end
        return A
    end

#Generate Sparse Matrix

function sparse_data_gen(m::Int64,n::Int64,S::Int64,σ::Float64)

    A = gen_GaussianMat(m,n) # generate a nomalized m by n matrix
    items = [j for j= 1:n]
    wv = [1 for j = 1:n]
    w = σ*randn(Float64,m,1) # generate the noise with variance σ
    # sparse support, or index.
    x = zeros(Float64,n)
    sparseSupport = sample( items , Weights(wv) , S, replace = false) # pick S random position for the non zero elements 
    randnTemp = randn(S, 1)                                           # of x
    x[sparseSupport] = randnTemp #fill x with all the non-zero elements
    
    y = A*x + w

    return y,A,x,sparseSupport 
end

function supp_of_H(vector, s) # function that returns the indices of s values with the highest norm
    H=[1] 
    if s > 1 
        for i in 2:s 
            append!(H,i) 
        end 
    end 
    small_vector = vector[1:s] # we assume the first s values in the vector have the highest norm
    for i in (s+1):length(vector) # starting from s+1 we check if there is another element with a higher norm
        if abs(vector[i]) > minimum(abs.(small_vector)) 
            H[argmin(abs.(small_vector))] = i # replace the index 
            small_vector[argmin(abs.(small_vector))] = vector[i] # replace the value 
        end 
    end
    return H 
end


supp_of_H (generic function with 1 method)

In [2]:
function rec_ista(x0,y,A,λ,τ,n,max_iter,stop_crit)

    #Define proximity operator
    #soft thresholding, y is a vector
    prox_l1(y,γ) = max.(abs.(y) - γ*ones(size(y)),0).*sign.(y);  

    f(x) = λ*norm(x,1) + (norm(y-A*x)).^2

    #gradient_h
    #this gradient_h is used for updating x
    grad_h(A,y,x) = transpose(A)*(A*x - y)
    
    #initialization
    x_cur = zeros(size(x0))
    x_next = zeros(size(x0));
    x_sol = zeros(size(x_cur))
    iter = 0

    ## main loop
    while iter < max_iter

        #gradient h
        #a forward step, for updating x
        r = x_cur - τ * grad_h(A,y,x_cur)
        r = reshape(r,n)

        #proximal operator of l1
        x_next = prox_l1(r,λ*τ)
        #calculate current f value
        f_cur = f(x_cur)
        #calculate next f value
        f_next = f(x_next)
        #calculate gap between f_cur and f_next
        error = f_next - f_cur 
        
        #break condition
            if (norm((x_next-x_cur),2)<stop_crit)||(norm(error)/norm(f_cur)<stop_crit)
                x_sol = x_next
                return x_sol, error, iter
                break
            end

        iter = iter + 1
        x_cur = x_next
    end   
    

end


rec_ista (generic function with 1 method)

In [3]:
#signal generation
m,n,S,σ = 32,64,8,1e-2
y,A,x_origin,sparseSupport = sparse_data_gen(m,n,S,σ);

x0 = zeros(size(x_origin))

#  find τ bounds
U,Sv,Vt = svd(transpose(A)*A)
lamda_max = maximum(Sv)
τ_upper = 2/(lamda_max)


0.3523436612274402

In [4]:
#τ--the step size
#τ = τ_upper/3
τ = 0.01
λ = 1e-2 

#options
max_iter = 100000
stop_crit = 1e-7

x_rebuilt, f_gap, iteration = rec_ista(x0,y,A,λ,τ,n,max_iter,stop_crit)

Supp_x_rebuilt = supp_of_H(x_rebuilt,S) 

#original supportset and rebuilt supportset
@show sort(Supp_x_rebuilt)
@show sort(sparseSupport);

#original signal and rebuilt signal
@show x_rebuilt;
@show x_origin;

#error between two signals
@show norm(x_origin - x_rebuilt);
@show iteration

sort(Supp_x_rebuilt) = [6, 13, 18, 21, 23, 38, 46, 51]
sort(sparseSupport) = [6, 13, 18, 21, 23, 38, 46, 51]
x_rebuilt = [0.0, 0.0, -0.0, 0.0, 0.0, -0.9069821733015826, -0.0, 0.0, 0.00593313667147943, 0.0, 0.0, -0.0, 0.7145958851848304, 0.0216757475350265, 0.0, 0.0, -0.0, 1.063377542318859, -0.0, 0.0, -2.1182052740566335, -0.024202037171226855, -0.16927171113102957, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.007689128965483521, 0.0, 0.0, -0.007976431113765973, -0.009793580563681866, 0.0, -0.0, 0.7382733729234541, 0.00110835489283321, 0.0, 0.0, -0.011131492545257119, 0.026361753636271912, 0.0, 0.0, 0.5318120244540697, -0.0331355260463475, -0.0, -0.0, 0.0015400363610407549, -0.7951835604775653, -0.01459617667745898, -0.0, -0.0, 0.0, 0.0, 0.0, -0.021784359187994224, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0]
x_origin = [0.0, 0.0, 0.0, 0.0, 0.0, -0.9078001422086724, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7342134696719567, 0.0, 0.0, 0.0, 0.0, 1.1044865146850764, 0.0, 0.0, -2.15083952312961, 0.0, -0.20717474

46515

In [5]:
#τ--the step size
τ = τ_upper-0.01
λ = 1e-2 

#options
max_iter = 10000
stop_crit = 1e-7

x_rebuilt, f_gap, iteration = rec_ista(x0,y,A,λ,τ,n,max_iter,stop_crit)

Supp_x_rebuilt = supp_of_H(x_rebuilt,S) 

#original supportset and rebuilt supportset
@show sort(Supp_x_rebuilt)
@show sort(sparseSupport);

#original signal and rebuilt signal
@show x_rebuilt;
@show x_origin;

#error between two signals
@show norm(x_origin - x_rebuilt);
@show iteration

sort(Supp_x_rebuilt) = [6, 13, 18, 21, 23, 38, 46, 51]
sort(sparseSupport) = [6, 13, 18, 21, 23, 38, 46, 51]
x_rebuilt = [0.0, 0.0, -0.0, 0.0, 0.0, -0.9069284031156662, -0.0, 0.0, 0.006020129485462563, 0.0, 0.0, -0.0, 0.7164997634174161, 0.021204731101228128, 0.0, 0.0, -0.0, 1.065998392790719, -0.0, 0.0, -2.1199950447571245, -0.022262079096950233, -0.1706604577843586, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.007202755687989714, 0.0, 0.0, -0.007863817256952857, -0.008599548076146126, 0.0, -0.0008339959262613126, 0.743525944546824, 0.0014209766532868626, 0.0, 0.0008045775336992674, -0.008210376670983146, 0.02453783606983662, 0.0, 0.0, 0.5326677695747216, -0.02956774488043089, -0.0, -0.0, 0.0009883584683034025, -0.7961953252248531, -0.013584806885121514, -0.0, -0.0, 0.0, 0.0, 0.0, -0.020866276071094845, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0]
x_origin = [0.0, 0.0, 0.0, 0.0, 0.0, -0.9078001422086724, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7342134696719567, 0.0, 0.0, 0.0, 0.0, 1.1044865146850764, 0.0, 

1721

In [6]:
# function check_correct_bit(m,n,σ,total,max_iter,stop_crit,check_crit)
    
#     Sparsity = collect(4:2:20)
#     correct_bit_l1 = zeros(length(Sparsity))
#     #options
    
#         for j = 1:length(Sparsity)
#             s_check = Sparsity[j]
#             success_l1 = 0
           
#             for i = 1:total
#                 #signal sparse_sig_generation
#                 y_check,A_check,x_origin,sparseSupport = sparse_data_gen(m,n,s_check,σ)

#                 #  find τ bounds
#                 U,Sv,Vt = svd(transpose(A_check)*A_check)
#                 lamda_max = maximum(Sv)
#                 τ_upper = 2/(lamda_max)

#                 #τ--the step size
#                 #τ = τ_upper/3

#                 τ_check = τ_upper/7

#                 λ = 1e-2 


#                 #two MM algorithms 
#                 x_sol_l1,~ = rec_ista(x_origin,y_check,A_check,λ,τ_check,n,max_iter,stop_crit);
#                Supp_x_sol_l1 = supp_of_H(x_sol_l1,s_check);

#                 if ((norm(x_origin-x_sol_l1,2) < check_crit))&&(sort(Supp_x_sol_l1)==sort(sparseSupport))
#                     success_l1 += 1
#                 end

            
#             end
#             correct_bit_l1[j] = success_l1/total
        
#         end

#         return correct_bit_l1
#         # return correct_bit_l1
# end





In [7]:
# m,n,σ = 32,64,1e-2 # characteristics of checked matrices 
# total = 50 # number of iterations of each  
# #correct_bit_l1= check_correct_bit(m,n,σ,total)
# max_iter,stop_crit,check_crit = 1000000, 1e-7,1e-2
# cor_rate_l1= check_correct_bit(m,n,σ,total,max_iter,stop_crit,check_crit)
# #     @save "check_algorithms.jld2" correct_bit_l0_in correct_bit_l0_without_in correct_bit_omp correct_bit_sp #overwrite the existing saved values 
# # else
# #    @load "check_algorithms.jld2" correct_bit_l0_in correct_bit_l0_without_in correct_bit_omp correct_bit_sp  
# # end


In [8]:
# using  Plots
# S = collect(4:2:20)
# plot(S,cor_rate_l1,title = "Comparison of algorithms",xlabel = "Sparsity S",
# ylabel = "Avrg correct rate",xlims = (0, 20),lab = "Lasso_L1")
# # plot!(S,correct_bit_l0_without_in,lab = "MM_L0_without_indicator")
# # plot!(S,correct_bit_omp,lab = "omp")
# # plot!(S,correct_bit_sp,lab = "SP")

In [9]:
#Choose τ
function choose_τ(m,n,σ,S,length_τ, total,max_iteration,stop_crit)
    
    τ_list = zeros(length_τ)
    avg_τ = zeros(length_τ)
    
    #find average value of τ
    for i = 1:total

        #initialization of norm_list
        norm_list = zeros(size(avg_τ))

        #signal sparse_sig_generation  
        y_ch,A_ch,x_origin,sparseSupport = sparse_data_gen(m,n,S,σ)
        x0 = zeros(size(x_origin))

        #  find τ bounds
        U,Sv,Vt = svd(transpose(A_ch)*A_ch)
        lamda_max = maximum(Sv)
        τ_upper = 2/(lamda_max)
    
        #find a suitable τ
        for j = 1:length_τ

            #τ--the step size
            #ex: τ = τ_upper/3
            τ_ch = τ_upper/(length_τ + 1 - j)
            λ = 1e-2 
            τ_list[j] += τ_ch
            #l1 algorithms 
            x_sol_τ_ch,gap,iteration = rec_ista(x0,y_ch,A_ch,λ,τ_ch,n,max_iteration,stop_crit);
            #Supp_x_sol_τ_ch = supp_of_H(x_sol_τ_ch,S);

            norm_list[j] += norm(x_origin-x_sol_τ_ch,2)

        end
        
        avg_τ += norm_list
        
    end
    avg_τ = avg_τ/total
    τ_list = τ_list/total
    return avg_τ,τ_list
        
end



choose_τ (generic function with 1 method)

In [10]:
#plots
using Plots
m,n,S,σ = 32,64,8,1e-2 # characteristics of checked matrices
max_iter,stop_crit = 100000, 1e-7
length_τ = 30
#total = 5
total = 50
avg_norm_list,τ_list = choose_τ(m,n,σ,S,length_τ,total,max_iter,stop_crit)

([0.11527971337955868, 0.11519132840925775, 0.11507888896915949, 0.11498638992938787, 0.11489137896296092, 0.11478661603601123, 0.1146907481963178, 0.11459377859914856, 0.11439707989730474, 0.11431443109522582  …  0.11332048887856697, 0.11318922441187547, 0.11308312326133896, 0.11297048276909402, 0.11285521084900114, 0.11272461583661836, 0.11258937034013122, 0.11165704072368447, 0.11095941673509016, 0.11068500253948592], [0.01280619345797414, 0.013247786335835317, 0.013720921562115159, 0.014229103842193488, 0.01477637706689325, 0.015367432149568972, 0.01600774182246768, 0.016703730597357577, 0.01746299107905565, 0.018294562082820213  …  0.03841858037392243, 0.04268731152658048, 0.04802322546740303, 0.054883686248460635, 0.06403096728987072, 0.07683716074784486, 0.09604645093480606, 0.12806193457974144, 0.19209290186961211, 0.38418580373922423])

In [11]:
plot(τ_list,avg_norm_list,label = ["avg norm between x_origin and x_rebuilt"], lw = 2)

# #total = 10
# total = 10
# avg_norm_list,τ_list = choose_τ(m,n,σ,S,length_τ,total,max_iter,stop_crit)
# plot!(τ_list,avg_norm_list,label = ["10 iterations"], lw = 2)

# #total = 15
# total = 15
# avg_norm_list,τ_list = choose_τ(m,n,σ,S,length_τ,total,max_iter,stop_crit)
# plot!(τ_list,avg_norm_list,label = ["15 iterations"], lw = 2)

# #total = 20
# total = 20
# avg_norm_list,τ_list = choose_τ(m,n,σ,S,length_τ,total,max_iter,stop_crit)
# plot!(τ_list,avg_norm_list,label = ["20 iterations"], lw = 2)

xlabel!("τ")
ylabel!(" norm(x_origin-x_rebuilt,2) ")
title!("τ VS norm")
savefig("τ VS norm");


## Comparision between SP and ISTA

In [12]:
# SP implementation
using Distributions
using Random
using JLD2

function subspace_pursuit(y,A,s)
    T = supp_of_H((A')*y,s) # take an initial T set of indices
    T = sort(T) 
    A_T = A[:,T] # generate the sparse matrix A
    y_res = y - A_T*pinv(A_T'*A_T)*A_T'*y # calculate the initial y residue
    iterations = 0
    xl = []
    while true
        iterations += 1
        y_res_last = y_res # store the previous y residue
        T_new = supp_of_H((A')*y_res,s) # get another set of s
        T_union = union(T,T_new) # combine them to get a set of 2s (expand support)
        T_union = sort(T_union)
        A_T = A[:,T_union] # generate a 2s sparse matrix A
        b = pinv(A_T'*A_T)*A_T'*y # estimate a 2s sparse signal
        T = supp_of_H(b,s) # shrink support
        T = T[1:s]
        T = T_union[T]
        A_T = A[:,sort(T)]
        xl = pinv(A_T'*A_T)*A_T'*y # estimate s sparse signal
        y_res = y - A_T*xl # compute estimation error
        if ((abs((norm(y_res_last,2)-norm(y_res,2)))<1e-100)||(norm(y_res,2)==0 || iterations > 100))
            break
        end
    end
    x_final=zeros(1,length(A[1,:]))
    x_final[1,sort(T)]=xl 
    return (x_final')
end


subspace_pursuit (generic function with 1 method)

In [13]:
subspace_pursuit(y,A,s)

LoadError: UndefVarError: s not defined

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.